# Notebook 1: Setting up the Environment

## JAX and Keras 3 Setup

GravyFlow now uses Keras 3 with the JAX backend. This notebook guides you through setting up the environment variables and verifying the installation.

## Notebook imports

First, we set the `KERAS_BACKEND` environment variable to `jax` **before** importing Keras or GravyFlow. This ensures Keras uses JAX as its backend.

In [1]:
import os

# Set the backend to JAX
os.environ["KERAS_BACKEND"] = "jax"

import keras
import jax

print(f"Keras version: {keras.__version__}")
print(f"Keras backend: {keras.backend.backend()}")
print(f"JAX devices: {jax.devices()}")

Keras version: 3.12.0
Keras backend: jax
JAX devices: [CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3), CudaDevice(id=4), CudaDevice(id=5), CudaDevice(id=6), CudaDevice(id=7)]


W1206 19:42:02.785263 1558454 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1206 19:42:02.785319 1558457 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1206 19:42:02.785466 1558451 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1206 19:42:02.785581 1558453 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1206 19:42:02.785664 1558452 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1206 19:42:02.793959 1558450 cuda_executor.cc:1802] GPU int

## Import GravyFlow

Next, we will import the GravyFlow module.

In [2]:
import gravyflow as gf

## GPU Configuration

JAX automatically detects and uses available GPUs. You can control which GPUs are visible to JAX using the `CUDA_VISIBLE_DEVICES` environment variable before starting the python process or notebook kernel.

For example, to use only the second GPU:
```bash
export CUDA_VISIBLE_DEVICES=1
```

In a notebook, you can check the active devices using `jax.devices()`.

Now that the environment is set up, let's move on to acquiring our first interferometer noise background dataset.